In [ ]:
!pip install ../input/sacremoses/sacremoses-master/ > /dev/null

In [ ]:
!pip install "../input/kerasswa/keras-swa-0.1.2"  > /dev/null

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import gc
import glob
import os
import sys
import string
import random
from tqdm import tqdm_notebook
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder, minmax_scale, MultiLabelBinarizer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import MultiTaskElasticNet
from scipy.stats import spearmanr, rankdata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # -1 = CPU only

import nltk
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

from gensim.models.doc2vec import TaggedDocument
from gensim import utils
import torch

sys.path.insert(0, "../input/transformers/transformers-master/")
import transformers


import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import tensorflow_hub as hub

import keras
import keras.backend as K
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras import Model
from swa.keras import SWA

import lightgbm as lgb
import catboost as cb
import xgboost as xgb

import pickle    

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
                
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
nltk.data.path.append("../input/nltk-data/nltk_data")
# thse are in nltk_data dataset 
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
# nltk.download('stopwords')


In [ ]:
#INPUT_PATH="/kaggle/input/"
INPUT_PATH="../input/"
train = pd.read_csv(INPUT_PATH+'google-quest-challenge/train.csv')
test = pd.read_csv(INPUT_PATH+'google-quest-challenge/test.csv')
submission = pd.read_csv(INPUT_PATH+'google-quest-challenge/sample_submission.csv')

In [ ]:
targets = [
        'question_asker_intent_understanding',
        'question_body_critical',
        'question_conversational',
        'question_expect_short_answer',
        'question_fact_seeking',
        'question_has_commonly_accepted_answer',
        'question_interestingness_others',
        'question_interestingness_self',
        'question_multi_intent',
        'question_not_really_a_question',
        'question_opinion_seeking',
        'question_type_choice',
        'question_type_compare',
        'question_type_consequence',
        'question_type_definition',
        'question_type_entity',
        'question_type_instructions',
        'question_type_procedure',
        'question_type_reason_explanation',
        'question_type_spelling',
        'question_well_written',
        'answer_helpful',
        'answer_level_of_information',
        'answer_plausible',
        'answer_relevance',
        'answer_satisfaction',
        'answer_type_instructions',
        'answer_type_procedure',
        'answer_type_reason_explanation',
        'answer_well_written'    
    ]

input_columns = ['question_title','question_body','answer']



In [ ]:
#clean data
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}


def clean_text(x):
    x = str(x).replace("\n","")
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_data(df, columns: list):
    for col in columns:
#         df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = df[col].apply(lambda x: clean_text(x.lower()))
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))

    return df


In [ ]:
train = clean_data(train, input_columns)
test = clean_data(test, input_columns)

In [ ]:
train['question_body'][0]

In [ ]:
def constructLabeledSentences(data):
    sentences=[]
    for index, row in data.iteritems():
        sentences.append(TaggedDocument(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))
    return sentences

def textClean(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]    
    text = " ".join(text)
    return(text)
    
def cleanup(text):
    text = textClean(text)
    text= text.translate(str.maketrans("","", string.punctuation))
    return text

train_question_body_sentences = constructLabeledSentences(train['question_body'])
train_question_title_sentences = constructLabeledSentences(train['question_title'])
train_answer_sentences = constructLabeledSentences(train['answer'])

test_question_body_sentences = constructLabeledSentences(test['question_body'])
test_question_title_sentences = constructLabeledSentences(test['question_title'])
test_answer_sentences = constructLabeledSentences(test['answer'])

In [ ]:
from gensim.models import Doc2Vec

all_sentences = train_question_body_sentences + \
                train_answer_sentences + \
                test_question_body_sentences + \
                test_answer_sentences

Text_INPUT_DIM=128
text_model = Doc2Vec(min_count=1, window=5, vector_size=Text_INPUT_DIM, sample=1e-4, negative=5, workers=4, epochs=5,seed=1)
text_model.build_vocab(all_sentences)
text_model.train(all_sentences, total_examples=text_model.corpus_count, epochs=text_model.iter)

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer

def normalize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN') or tag.startswith('PRP'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            continue
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos).lower())
    return lemmatized_sentence

#print(set(normalize_sentence(word_tokenize(train['question_body'][0]))))

In [ ]:
def normalize_vectorize(df, columns: list):
    for col in columns:
        print(col)
        df[col+'_norm'] = df[col].apply(lambda x: ' '.join(set(normalize_sentence(word_tokenize(x)))))
        df[col+'_vec'] = df[col].apply(lambda x: text_model.infer_vector([x]))

    return df

train = normalize_vectorize(train, input_columns)
test = normalize_vectorize(test, input_columns)


In [ ]:
#bert embedings
try:
    pbe = load_obj("../input/questembeddings/precomputed_bert_embeddings")
    train_question_body_dense = pbe['train_question_body_dense']
    train_answer_dense = pbe['train_answer_dense']
    train_question_title_dense = pbe['train_question_title_dense']
    test_question_body_dense = pbe['test_question_body_dense']
    test_answer_dense = pbe['test_answer_dense']
    test_question_title_dense = pbe['test_question_title_dense']
except:
    print("Load failed, build embedding")
    def sigmoid(x):
        return 1 / (1 + math.exp(-x))

    def chunks(l, n):
        """Yield successive n-sized chunks from l."""
        for i in range(0, len(l), n):
            yield l[i:i + n]

    def fetch_vectors(string_list, batch_size=64):
        # inspired by https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
        DEVICE = torch.device("cuda")
        tokenizer = transformers.DistilBertTokenizer.from_pretrained("../input/distilbertbaseuncased/")
        model = transformers.DistilBertModel.from_pretrained("../input/distilbertbaseuncased/")
        model.to(DEVICE)

        fin_features = []
        for data in tqdm_notebook(chunks(string_list, batch_size)):
            tokenized = []
            for x in data:
                x = " ".join(x.strip().split()[:300])
                tok = tokenizer.encode(x, add_special_tokens=True)
                tokenized.append(tok[:512])

            max_len = 512
            padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized], dtype='int64')
            attention_mask = np.where(padded != 0, 1, 0)
            input_ids = torch.tensor(padded).to(DEVICE)
            attention_mask = torch.tensor(attention_mask).to(DEVICE)

            with torch.no_grad():
                last_hidden_states = model(input_ids, attention_mask=attention_mask)

            features = last_hidden_states[0][:, 0, :].cpu().numpy()
            fin_features.append(features)

        fin_features = np.vstack(fin_features)
        return fin_features

    train_question_body_dense = fetch_vectors(train.question_body.values)
    train_answer_dense = fetch_vectors(train.answer.values)
    train_question_title_dense = fetch_vectors(train.question_title.values)


    test_question_body_dense = fetch_vectors(test.question_body.values)
    test_answer_dense = fetch_vectors(test.answer.values)
    test_question_title_dense = fetch_vectors(test.question_title.values)

    precomputed_bert_embeddings = {
        'train_question_body_dense': train_question_body_dense,
        'train_answer_dense': train_answer_dense,
        'train_question_title_dense': train_question_title_dense,
        'test_question_body_dense': test_question_body_dense,
        'test_answer_dense': test_answer_dense,
        'test_question_title_dense': test_question_title_dense,

    }

    #save_obj(precomputed_bert_embeddings,"../input/questembeddings/precomputed_bert_embeddings")

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))
tsvd = TruncatedSVD(n_components = 128, n_iter=5)
tfquestion_title = tfidf.fit_transform(train["question_title"].values)
tfquestion_title_test = tfidf.transform(test["question_title"].values)
tfquestion_title = tsvd.fit_transform(tfquestion_title)
tfquestion_title_test = tsvd.transform(tfquestion_title_test)

tfquestion_body = tfidf.fit_transform(train["question_body"].values)
tfquestion_body_test = tfidf.transform(test["question_body"].values)
tfquestion_body = tsvd.fit_transform(tfquestion_body)
tfquestion_body_test = tsvd.transform(tfquestion_body_test)

tfanswer = tfidf.fit_transform(train["answer"].values)
tfanswer_test = tfidf.transform(test["answer"].values)
tfanswer = tsvd.fit_transform(tfanswer)
tfanswer_test = tsvd.transform(tfanswer_test)

In [ ]:
torch.cuda.empty_cache() # release all gpu memory from pytorch

In [ ]:
# universal sentence encoder

try:
    embeddings_train = load_obj("../input/questembeddings/use_embeddings_train")
    embeddings_test = load_obj("../input/questembeddings/use_embeddings_test")
except:
    print("Load failed, build embedding")
    try:
        module_url = INPUT_PATH+'universalsentenceencoderlarge4/'
        embed = hub.load(module_url)
        def UniversalEmbedding(x):
            results = embed(tf.squeeze(tf.cast(x, tf.string)))["outputs"]
            return keras.backend.concatenate([results])
    except:
        module_url = INPUT_PATH+'universalsentenceencoderlarge3/'
        embed = hub.Module(module_url)
        def UniversalEmbedding(x):
            results = embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
            return keras.backend.concatenate([results])

    embeddings_train = {}
    embeddings_test = {}
    for text in input_columns:
        print(text)
        train_text = train[text].str.replace('?', '.').str.replace('!', '.').tolist()
        test_text = test[text].str.replace('?', '.').str.replace('!', '.').tolist()

        curr_train_emb = []
        curr_test_emb = []
        batch_size = 4
        ind = 0
        while ind*batch_size < len(train_text):
            curr_train_emb.append(embed(train_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
            ind += 1

        ind = 0
        while ind*batch_size < len(test_text):
            curr_test_emb.append(embed(test_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
            ind += 1    

        embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
        embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)

    del embed
    K.clear_session()
    gc.collect()
    
    #save_obj(embeddings_train,"../input/questembeddings/use_embeddings_train")
    #save_obj(embeddings_test,"../input/questembeddings/use_embeddings_test")

In [ ]:
find = re.compile(r"^[^.]*")
train['netloc'] = train['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
test['netloc'] = test['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

features_lrg = ['category', 'netloc', 'question_user_name','answer_user_name','host']
features_sml = ['category', 'netloc', 'host']
    
features_train = []
features_test = []
for feature in features_sml:
    merged = pd.concat([train[feature], test[feature]]).to_numpy().reshape(-1, 1).squeeze()
    encoded_size = len(max(merged, key=len))
    print(feature,encoded_size)
    ctr = 0
    column = []
    for val in train[feature].values:
        ctr +=1
        output = np.zeros(encoded_size)
        output[:len(val)] = [ord(i)/128. for i in val]
        column.append(output)
    features_train.append(column)
        
    column = []
    for val in test[feature].values:
        output = np.zeros(encoded_size)
        output[:len(val)] = [ord(i)/128. for i in val]
        column.append(output)
    features_test.append(column)

features_train_lrg = []
features_test_lrg = []
for feature in features_lrg:
    merged = pd.concat([train[feature], test[feature]]).to_numpy().reshape(-1, 1).squeeze()
    encoded_size = len(max(merged, key=len))
    print(feature,encoded_size)
    ctr = 0
    column = []
    for val in train[feature].values:
        ctr +=1
        output = np.zeros(encoded_size)
        output[:len(val)] = [ord(i)/128. for i in val]
        column.append(output)
    features_train.append(column)
        
    column = []
    for val in test[feature].values:
        output = np.zeros(encoded_size)
        output[:len(val)] = [ord(i)/128. for i in val]
        column.append(output)
    features_test.append(column)
    
    
l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)
cos_dist = lambda x, y: (x*y).sum(axis=1)
abs_dist = lambda x, y: np.abs(x-y).sum(axis=1)
sum_dist = lambda x, y: (x+y).sum(axis=1)

dist_features_train = np.array([
    l2_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    l2_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    l2_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding']),
    l2_dist(np.array([x for x in train.question_body_vec.values]), np.array([x for x in train.answer_vec.values])),
    l2_dist(np.array([x for x in train.question_title_vec.values]), np.array([x for x in train.answer_vec.values])),
    l2_dist(np.array([x for x in train.question_body_vec.values]), np.array([x for x in train.question_title_vec.values])),
    cos_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    cos_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    cos_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding']),
    cos_dist(np.array([x for x in train.question_body_vec.values]), np.array([x for x in train.answer_vec.values])),
    cos_dist(np.array([x for x in train.question_title_vec.values]), np.array([x for x in train.answer_vec.values])),
    cos_dist(np.array([x for x in train.question_body_vec.values]), np.array([x for x in train.question_title_vec.values])),
    abs_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    abs_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    abs_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding']),
    abs_dist(np.array([x for x in train.question_body_vec.values]), np.array([x for x in train.answer_vec.values])),
    abs_dist(np.array([x for x in train.question_title_vec.values]), np.array([x for x in train.answer_vec.values])),
    abs_dist(np.array([x for x in train.question_body_vec.values]), np.array([x for x in train.question_title_vec.values])),
    sum_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    sum_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    sum_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding']),
    sum_dist(np.array([x for x in train.question_body_vec.values]), np.array([x for x in train.answer_vec.values])),
    sum_dist(np.array([x for x in train.question_title_vec.values]), np.array([x for x in train.answer_vec.values])),
    sum_dist(np.array([x for x in train.question_body_vec.values]), np.array([x for x in train.question_title_vec.values])),
    l2_dist(train_question_body_dense, train_answer_dense),
    cos_dist(train_question_body_dense, train_answer_dense),
    abs_dist(train_question_body_dense, train_answer_dense),
    sum_dist(train_question_body_dense, train_answer_dense),
    l2_dist(train_question_body_dense, train_question_title_dense),
    cos_dist(train_question_body_dense, train_question_title_dense),
    abs_dist(train_question_body_dense, train_question_title_dense),
    sum_dist(train_question_body_dense, train_question_title_dense),
    l2_dist(train_answer_dense, train_question_title_dense),
    cos_dist(train_answer_dense, train_question_title_dense),
    abs_dist(train_answer_dense, train_question_title_dense),
    sum_dist(train_answer_dense, train_question_title_dense),
]).T


dist_features_test = np.array([
    l2_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    l2_dist(np.array([x for x in test.question_body_vec.values]), np.array([x for x in test.answer_vec.values])),
    l2_dist(np.array([x for x in test.question_title_vec.values]), np.array([x for x in test.answer_vec.values])),
    l2_dist(np.array([x for x in test.question_body_vec.values]), np.array([x for x in test.question_title_vec.values])),
    cos_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    cos_dist(np.array([x for x in test.question_body_vec.values]), np.array([x for x in test.answer_vec.values])),
    cos_dist(np.array([x for x in test.question_title_vec.values]), np.array([x for x in test.answer_vec.values])),
    cos_dist(np.array([x for x in test.question_body_vec.values]), np.array([x for x in test.question_title_vec.values])),
    abs_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    abs_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    abs_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    abs_dist(np.array([x for x in test.question_body_vec.values]), np.array([x for x in test.answer_vec.values])),
    abs_dist(np.array([x for x in test.question_title_vec.values]), np.array([x for x in test.answer_vec.values])),
    abs_dist(np.array([x for x in test.question_body_vec.values]), np.array([x for x in test.question_title_vec.values])),
    sum_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    sum_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    sum_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    sum_dist(np.array([x for x in test.question_body_vec.values]), np.array([x for x in test.answer_vec.values])),
    sum_dist(np.array([x for x in test.question_title_vec.values]), np.array([x for x in test.answer_vec.values])),
    sum_dist(np.array([x for x in test.question_body_vec.values]), np.array([x for x in test.question_title_vec.values])),
    l2_dist(test_question_body_dense, test_answer_dense),
    cos_dist(test_question_body_dense, test_answer_dense),
    abs_dist(test_question_body_dense, test_answer_dense),
    sum_dist(test_question_body_dense, test_answer_dense),
    l2_dist(test_question_body_dense, test_question_title_dense),
    cos_dist(test_question_body_dense, test_question_title_dense),
    abs_dist(test_question_body_dense, test_question_title_dense),
    sum_dist(test_question_body_dense, test_question_title_dense),
    l2_dist(test_answer_dense, test_question_title_dense),
    cos_dist(test_answer_dense, test_question_title_dense),
    abs_dist(test_answer_dense, test_question_title_dense),
    sum_dist(test_answer_dense, test_question_title_dense),
]).T





In [ ]:
possible_features_train = [
    [item for k, item in embeddings_train.items()],
    features_train,
    features_train_lrg,
    [ dist_features_train ],
    [ [x for x in train.question_body_vec.values] ],
    [ [x for x in train.question_title_vec.values] ],
    [ [x for x in train.answer_vec.values] ],
    [ train_question_body_dense ],
    [ train_answer_dense ],
    [ tfquestion_title ],
    [ tfquestion_body ],
    [ tfanswer ]
    
]
possible_features_test = [
    [item for k, item in embeddings_test.items()],
    features_test,
    features_test_lrg,
    [ dist_features_test ],
    [ [x for x in test.question_body_vec.values] ],
    [ [x for x in test.question_title_vec.values] ],
    [ [x for x in test.answer_vec.values] ],
    [ test_question_body_dense ],
    [ test_answer_dense ],
    [ tfquestion_title_test ],
    [ tfquestion_body_test ],
    [ tfanswer_test ]
]

def get_train_test(split=0.8):
    total_len = len(possible_features_train)
    r_idx = random.sample(range(total_len), int(total_len * split))
    
    train = [ train_question_title_dense ]

    test =  [ test_question_title_dense ]

    for i in r_idx:
        train += possible_features_train[i]
        test += possible_features_test[i]
        
    return np.hstack(train),np.hstack(test)

X_train,X_test = get_train_test()
y_train = train[targets].values

In [ ]:
from keras.losses import *
def bce(t,p):
    return binary_crossentropy(t,p)

def custom_loss(true,pred):
    bce = binary_crossentropy(true,pred)
    return bce + logcosh(true,pred)

def swish(x):
    return K.sigmoid(x) * x

def relu1(x):
    return keras.activations.relu(x, alpha=0.0, max_value=1., threshold=0.0)

def create_model1(X_train):
    input1 = Input(shape=(X_train.shape[1],))
    x = Dense(64, activation='elu',kernel_initializer='lecun_normal')(input1)
    x = Dense(128, activation='elu',kernel_initializer='lecun_normal')(x)
    x = Dropout(0.4)(x)
    output = Dense(len(targets),activation='sigmoid',name='output')(x)
    model = Model(inputs=input1, outputs=output)
    return model


def create_model2(X_train):
    input1 = Input(shape=(X_train.shape[1],))
    x = Dense(512, activation='elu',kernel_initializer='lecun_normal')(input1)
    x = Dense(256, activation='elu',
              kernel_initializer='lecun_normal', 
              #kernel_regularizer=keras.regularizers.l2(0.01)
             )(x)
    output = Dense(len(targets),activation='sigmoid',name='output')(x)
    model = Model(inputs=input1, outputs=output)
    return model


def create_model3(X_train):
    input1 = Input(shape=(X_train.shape[1],))
    x = Dense(200, activation='selu',kernel_initializer='lecun_normal')(input1)
    x = Dense(512, activation='selu',
              kernel_initializer='lecun_normal', 
              )(x)
    x = Dropout(0.4)(x)
    output = Dense(len(targets),activation='sigmoid',name='output')(x)
    model = Model(inputs=input1, outputs=output)
    return model

def create_model4(X_train):
    input1 = Input(shape=(X_train.shape[1],))
    x = Dense(512, activation='elu',
              kernel_initializer='lecun_normal', 
              )(input1)
    x = Dropout(0.2)(x)
    output = Dense(len(targets),activation='sigmoid',name='output')(x)
    model = Model(inputs=input1, outputs=output)
    return model

def create_model5(X_train):
    input1 = Input(shape=(X_train.shape[1],))
    x = Dense(4096, activation='selu',kernel_initializer='lecun_normal')(input1)
    x = Dense(512, activation='selu',
              kernel_initializer='lecun_normal', 
              kernel_regularizer=keras.regularizers.l2(0.01)
             )(x)
    output = Dense(len(targets),activation='sigmoid',name='output')(x)
    model = Model(inputs=input1, outputs=output)
    return model

def create_model(X_train):
    model = random.choice([create_model1, create_model2, create_model3, create_model4, create_model5])(X_train)
    model.summary()
    return model


In [ ]:
import gc
print(gc.collect())

In [ ]:
def pearson_metric(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    # reshape stage
#     y_true = K.reshape(y_true, shape=(-1, 224 * 224 * 3))
#     y_pred = K.reshape(y_pred, shape=(-1, 224 * 224 * 3))
    # normalizing stage - setting a 0 mean.
    y_true -= K.mean(y_true)
    y_pred -= K.mean(y_pred)
    # normalizing stage - setting a 1 variance
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    # final result
    pearson_correlation = K.sum(y_true * y_pred, axis=-1)
    return 1-pearson_correlation

    

In [ ]:
# Compatible with tensorflow backend
error_pred_y = None
error_y = None
class SpearmanRhoCallback(Callback):
    def __init__(self, training_data, validation_data, patience, model_name, reload=False):
        global noise
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        
        self.patience = patience
        self.value = -1
        self.bad_epochs = 0
        self.model_name = model_name
        self.reload = reload

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        y_pred_val = self.model.predict(self.x_val)
        noise = np.random.normal(0, 1e-7, y_pred_val.shape[0])
        rho_val = np.mean([spearmanr(self.y_val[:, ind] + noise, y_pred_val[:, ind] + noise).correlation for ind in range(y_pred_val.shape[1])])
        print('\r  val_spearman-rho: %s' % (str(round(rho_val, 4))), end=100*' '+'\n')

        if rho_val >= self.value:
            self.model.save_weights(self.model_name)
            self.value = rho_val
        else:
            self.model.load_weights(self.model_name)
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        global error_pred_y, error_y        
        y_pred_val = self.model.predict(self.x_val)
        noise = np.random.normal(0, 1e-7, y_pred_val.shape[0])
        rho_val = np.mean([spearmanr(self.y_val[:, ind] + noise, y_pred_val[:, ind] + noise).correlation for ind in range(y_pred_val.shape[1])])
        if np.isnan(rho_val):
            print("Error")
            error_pred_y = y_pred_val
            error_y = self.y_val
            print(y_pred_val)
            print(self.x_val)
            print(self.y_val)
            raise "bogus error"
#         y_pred = self.model.predict(self.x)
#         rho = np.mean([spearmanr(self.y[:, ind], y_pred[:, ind] + np.random.normal(0, 1e-7, y_pred.shape[0])).correlation for ind in range(y_pred.shape[1])])
        if rho_val >= self.value:
            self.model.save_weights(self.model_name)
            self.value = rho_val
        else:
            self.bad_epochs += 1
#         if self.bad_epochs >= self.patience:
#             print("Epoch %05d: early stopping Threshold" % epoch)
#             self.model.stop_training = True
        print('\r  val_spearman-rho: %s' % (str(round(rho_val, 4))), end=100*' '+'\n')
#         print('\rtrain_spearman-rho: %s' % (str(round(rho, 4))), end=100*' '+'\n')
        if self.reload:
            print('  reload best: %s' % (str(round(self.value, 4))))
            self.model.load_weights(self.model_name)
        return rho_val

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [ ]:
all_predictions = []
model_idx =0 
def run_model():
    global y_train,all_predictions, model_idx
    X_train,X_test = get_train_test()
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=7, min_lr=1e-6, verbose=1)

    early_stop = EarlyStopping(monitor='val_loss',
                                  min_delta=0,
                                  patience=15,
                                  mode='auto')

    kf = KFold(n_splits=5, random_state=random.randint(0,1000), shuffle=True)

    for ind, (tr, val) in enumerate(kf.split(X_train)):
        X_tr = X_train[tr]
        y_tr = y_train[tr]
        X_vl = X_train[val]
        y_vl = y_train[val]
        model_idx+=1
        model = create_model(X_train)
        optimizer = Adam(lr=5e-4,clipnorm=1.4)
        model.compile(optimizer=optimizer, loss=custom_loss, metrics=[bce,logcosh])
        model.fit(
            X_tr, y_tr, 
            epochs=100, batch_size=64, 
            validation_data=(X_vl, y_vl), 
            verbose=True, 
            callbacks=[SpearmanRhoCallback(training_data=(X_tr, y_tr), 
                                           validation_data=(X_vl, y_vl),
                                           patience=10, 
                                           model_name=f'best_model_batch{model_idx}.h5',
                                           reload=True),
                       reduce_lr,
                       early_stop
                      ]
        )
        optimizer = SGD(lr=0.01,clipnorm=1.1)
        model.compile(optimizer=optimizer, loss=custom_loss, metrics=[bce,logcosh])
        history = model.fit(
            X_tr, y_tr, 
            epochs=50, batch_size=64, 
            validation_data=(X_vl, y_vl), 
            verbose=True, 
            callbacks=[SpearmanRhoCallback(training_data=(X_tr, y_tr), 
                                           validation_data=(X_vl, y_vl),
                                           patience=10, 
                                           model_name=f'best_model_batch_sgd{model_idx}.h5'),
                       reduce_lr,
                       early_stop   
                      ]
        )
        if min(history.history['val_bce']) < 0.375:
            all_predictions.append(model.predict(X_test))

    optimizer = Adam(lr=2e-4,clipnorm=1.9)
    model.compile(optimizer=optimizer, loss=bce)
    model.fit(
        X_train, y_train, 
        epochs=30, batch_size=128, 
        validation_data=(X_train, y_train), 
        verbose=True, 
        callbacks=[SpearmanRhoCallback(training_data=(X_train, y_train), 
                                       validation_data=(X_train, y_train),
                                       patience=10, 
                                       model_name=f'best_model_batchoverfit{model_idx}.h5',
                                       reload=False),
                   reduce_lr,
                   early_stop
                  ]
    )
    all_predictions.append(model.predict(X_test))


In [ ]:
all_predictions = []
while len(all_predictions) < 20:
    run_model()

In [ ]:
len(all_predictions)

In [ ]:
# This is slow...
# from sklearn.linear_model import MultiTaskElasticNet
# 
# kf = KFold(n_splits=5, random_state=2019, shuffle=True)
# for ind, (tr, val) in enumerate(kf.split(X_train)):
#     X_tr = X_train[tr]
#     y_tr = y_train[tr]
#     X_vl = X_train[val]
#     y_vl = y_train[val]
    
#     model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
#     model.fit(X_tr, y_tr)
#     all_predictions.append(model.predict(X_test))
    
# model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
# model.fit(X_train, y_train)
# all_predictions.append(model.predict(X_test))

In [ ]:
K.clear_session()
gc.collect()

In [ ]:
# Catboost didn't work well
# # clear session and clear cuda memory for catboost
# K.clear_session()
# gc.collect()

# from numba import cuda;
# cuda.select_device(0);
# cuda.close()


In [ ]:
# kf = KFold(n_splits=3, random_state=42, shuffle=True)
# from xgboost import XGBClassifier
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.multioutput import MultiOutputRegressor
# # all_predictions = []
# for ind, (tr, val) in enumerate(kf.split(X_train)):
#     X_tr = X_train[tr]
#     y_tr = y_train[tr]
#     X_vl = X_train[val]
#     y_vl = y_train[val]

#     print(X_tr.shape)
#     print(y_tr.shape)


    
#     gs1 = MultiOutputRegressor(
#         cb.CatBoostRegressor(iterations=400,
#                             learning_rate=0.7,
#                             depth=9,
#                             verbose = 20,
#                             task_type = 'GPU',
#                             loss_function = 'MAE')
        
#     )
#     gs1.fit(X_tr, y_tr);    

#     all_predictions.append(gs1.predict(X_test))
    


In [ ]:
test_preds = np.array([np.array([rankdata(c) for c in p.T]).T for p in all_predictions]).mean(axis=0)
max_val = test_preds.max() + 1
test_preds = test_preds/max_val + 1e-12

In [ ]:
submission = pd.read_csv(INPUT_PATH+'google-quest-challenge/sample_submission.csv')
submission[targets] = test_preds
submission.head(20)

In [ ]:
submission.to_csv("submission.csv", index = False)
